# Tools with MCP ⏰

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp-adapters`.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=<not set>
LANGSMITH_API_KEY=****2f66
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Loaded 5 MCP tools: ['add_time', 'compare_time', 'convert_timezone', 'current_time', 'relative_time']


Create an agent with the MCP-provided time tools.

In [3]:
import os
from langchain_ollama import ChatOllama

model = ChatOllama(model=os.getenv("OLLAMA_MODEL") , temperature= 0 , base_url=os.getenv("OLLAMA_BASE_URL"))

In [7]:
from langsmith import uuid7
run_id = uuid7()  

In [8]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model=model,
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

Ask about the current time in San Francisco.

In [9]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in SF right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What's the time in SF right now?
================================== Ai Message ==================================
Tool Calls:
  current_time (d4c7a179-43a0-44fe-a4b2-3eca17854df7)
 Call ID: d4c7a179-43a0-44fe-a4b2-3eca17854df7
  Args:
    format: RFC3339
    timezone: America/Los_Angeles
================================= Tool Message =================================
Name: current_time

2025-11-24T06:29:59-08:00
================================== Ai Message ==================================

The current time in San Francisco (America/Los_Angeles) is:

**2025‑11‑24T06:29:59‑08:00** (RFC 3339 format)
